# Quantum State Tomography

* **Last Updated:** Feb 28, 2019
* **Requires:** qiskit-terra 0.7

In [1]:
# Needed for functions
import numpy as np
import time

# Import Qiskit classes
import qiskit 
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer
from qiskit.quantum_info import state_fidelity
from qiskit.providers.aer import noise

# Tomography functions
import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
import qiskit.ignis.mitigation.measurement as mc

## 2-Qubit state tomography Example

In [2]:
# Create a state preparation circuit
q2 = QuantumRegister(2)
bell = QuantumCircuit(q2)
bell.h(q2[0])
bell.cx(q2[0], q2[1])
print(bell.qasm())

job = qiskit.execute(bell, Aer.get_backend('statevector_simulator'))
psi_bell = job.result().get_statevector(bell)
print(psi_bell)

OPENQASM 2.0;
include "qelib1.inc";
qreg q0[2];
h q0[0];
cx q0[0],q0[1];

[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


In [3]:
# Generate circuits and run on simulator
t = time.time()
qst_bell = state_tomography_circuits(bell, q2)
job = qiskit.execute(qst_bell, Aer.get_backend('qasm_simulator'), shots=5000)
print('Time taken:', time.time() - t)

# Extract tomography data so that countns are indexed by measurement configuration
# Note that the None labels are because this is state tomography instead of process tomography
# Process tomography would have the preparation state labels there

tomo_bell = StateTomographyFitter(job.result(), qst_bell)

Time taken: 0.35581493377685547


In [4]:
# Perform the tomography fit
rho_bell = tomo_bell.fit()
F_bell = state_fidelity(psi_bell, rho_bell)
print('Fit Fidelity =', F_bell)

Fit Fidelity = 0.9999687243422095


### Repeat the Example with Measurement Noise

In [5]:
#Add measurement noise
noise_model = noise.NoiseModel()
for qi in range(2):
    read_err = noise.errors.readout_error.ReadoutError([[0.75, 0.25],[0.1,0.9]])
    noise_model.add_readout_error(read_err,[qi])
    
#generate the calibration circuits
meas_calibs, state_labels = mc.complete_meas_cal(qr=q2)

backend = Aer.get_backend('qasm_simulator')
qobj_cal = qiskit.compile(meas_calibs, backend=backend, shots=15000)
qobj_tomo = qiskit.compile(qst_bell, backend=backend, shots=15000)

job_cal = backend.run(qobj_cal, noise_model=noise_model)
job_tomo = backend.run(qobj_tomo, noise_model=noise_model)

meas_fitter = mc.CompleteMeasFitter(job_cal.result(),state_labels)

tomo_bell = StateTomographyFitter(job_tomo.result(), qst_bell)

#no correction
rho_bell = tomo_bell.fit()
F_bell = state_fidelity(psi_bell, rho_bell)
print('Fit Fidelity (no correction) =', F_bell)

#correct data
correct_tomo_results = meas_fitter.filter.apply(job_tomo.result(), method='least_squares')
tomo_bell = StateTomographyFitter(correct_tomo_results, qst_bell)
rho_bell = tomo_bell.fit()
F_bell = state_fidelity(psi_bell, rho_bell)
print('Fit Fidelity (w/ correction) =', F_bell)

Fit Fidelity (no correction) = 0.5693532358477611
Fit Fidelity (w/ correction) = 0.9960235204067276


## Generating and fitting random states

We now test the functions on state generated by a circuit consiting of a layer of random single qubit unitaries u3

In [6]:
def random_u3_tomo(nq, shots):
    
    def rand_angles():
        return tuple(2 * np.pi * np.random.random(3) - np.pi)
    q = QuantumRegister(nq)
    circ = QuantumCircuit(q)
    for j in range(nq):
        circ.u3(*rand_angles(), q[j])
    job = qiskit.execute(circ, Aer.get_backend('statevector_simulator'))
    psi_rand = job.result().get_statevector(circ)
    
    qst_circs = state_tomography_circuits(circ, q)
    job = qiskit.execute(qst_circs, Aer.get_backend('qasm_simulator'),
                         shots=shots)
    tomo_data = StateTomographyFitter(job.result(), qst_circs)
    rho_cvx = tomo_data.fit(method='cvx')
    rho_lstsq = tomo_data.fit(method='lstsq')
    
    print('F fit (CVX) =', state_fidelity(psi_rand, rho_cvx))
    print('F fit (LSTSQ) =', state_fidelity(psi_rand, rho_lstsq))

In [7]:
for j in range(5):
    print('Random single-qubit unitaries: set {}'.format(j))
    random_u3_tomo(3, 5000)

Random single-qubit unitaries: set 0
F fit (CVX) = 0.9978156237448637
F fit (LSTSQ) = 0.9930491098490741
Random single-qubit unitaries: set 1
F fit (CVX) = 0.9981310819833952
F fit (LSTSQ) = 0.998209371153777
Random single-qubit unitaries: set 2
F fit (CVX) = 0.9986613929427925
F fit (LSTSQ) = 0.9977340937562109
Random single-qubit unitaries: set 3
F fit (CVX) = 0.9966797248885677
F fit (LSTSQ) = 0.9961833859762292
Random single-qubit unitaries: set 4
F fit (CVX) = 0.999391383434298
F fit (LSTSQ) = 0.9961654603890222


## 5-Qubit Bell State

In [8]:
# Create a state preparation circuit
q5 = QuantumRegister(5)
bell5 = QuantumCircuit(q5)
bell5.h(q5[0])
for j in range(4):
    bell5.cx(q5[j], q5[j + 1])

# Get ideal output state
job = qiskit.execute(bell5, Aer.get_backend('statevector_simulator'))
psi_bell5 = job.result().get_statevector(bell5)

# Generate circuits and run on simulator
t = time.time()
qst_bell5 = state_tomography_circuits(bell5, q5)
job = qiskit.execute(qst_bell5, Aer.get_backend('qasm_simulator'), shots=5000)

# Extract tomography data so that countns are indexed by measurement configuration
tomo_bell5 = StateTomographyFitter(job.result(), qst_bell5)
print('Time taken:', time.time() - t)

Time taken: 8.33616304397583


In [9]:
t = time.time()
rho_lstsq_bell5 = tomo_bell5.fit(method='lstsq')
print('Least-Sq Reconstruction')
print('Time taken:', time.time() - t)
print('Fit Fidelity:', state_fidelity(psi_bell5, rho_lstsq_bell5))

Least-Sq Reconstruction
Time taken: 5.282316207885742
Fit Fidelity: 0.9935772694475447


In [10]:
t = time.time()
rho_cvx_bell5 = tomo_bell5.fit(method='cvx')
print('CVX Reconstruction')
print('Time taken:', time.time() - t)
print('Fidelity:', state_fidelity(psi_bell5, rho_cvx_bell5))

CVX Reconstruction
Time taken: 57.68274211883545
Fidelity: 0.9999223159638952
